In [1]:
import xgboost as xgb
import tqdm as tq
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
from tqdm import tqdm
from xgboost import XGBRegressor
import numpy as np

### Train Data Preprocessing

In [5]:
train = pd.read_csv('train.csv')

In [69]:
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [6]:
train.pop('일조(hr)')
train.pop('일사(MJ/m2)')

train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')
train['month'] = train['일시'].dt.month
train['day'] = train['일시'].dt.weekday
train['hour'] = train['일시'].dt.hour
train['holiday'] = train.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
train['quarter'] = train.apply(lambda x: 2 if x['month']<7 else 3, axis=1)
train['sin_time'] = np.sin(2*np.pi*train.hour/24) # cyclical encoding
train['cos_time'] = np.cos(2*np.pi*train.hour/24)
train['DI'] = 1.8*train['기온(C)'] - 0.55*(1-train['습도(%)'])*(1.8*train['기온(C)']-26) + 32 # 불쾌지수

train.pop('num_date_time')
train.pop('일시')
train.pop('hour')
train['power'] = train['전력소비량(kWh)']
train.pop('전력소비량(kWh)')
train.pop('강수량(mm)')
train['풍속(m/s)'] = train['풍속(m/s)'].fillna(0)
train['습도(%)'] = train['습도(%)'].fillna(0)

In [71]:
train.to_csv('train_preprocessed.csv', index=False)

In [7]:
train.head()

,건물번호,기온(C),풍속(m/s),습도(%),month,day,holiday,quarter,sin_time,cos_time,DI,power
0,1,18.6,0.9,42.0,6,2,0,2,0.000000,1.000000,234.154,1085.28
1,1,18.0,1.1,45.0,6,2,0,2,0.258819,0.965926,219.280,1047.36
2,1,17.7,1.5,45.0,6,2,0,2,0.500000,0.866025,205.672,974.88
3,1,16.7,1.4,48.0,6,2,0,2,0.707107,0.707107,167.011,953.76
4,1,18.4,2.8,43.0,6,2,0,2,0.866025,0.500000,229.592,986.40


### Test Data Preprocessing

In [16]:
test = pd.read_csv('test.csv')

In [17]:
test['일시'] = pd.to_datetime(test['일시'], format='%Y%m%d %H')
test['month'] = test['일시'].dt.month
test['day'] = test['일시'].dt.weekday
test['hour'] = test['일시'].dt.hour
test['holiday'] = test.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
test['quarter'] = test.apply(lambda x: 2 if x['month']<7 else 3, axis=1)
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['DI'] = 1.8*test['기온(C)'] - 0.55*(1-test['습도(%)'])*(1.8*test['기온(C)']-26) + 32

test.pop('num_date_time')
test.pop('일시')
test.pop('hour')
test.pop('강수량(mm)')

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
16795    0.0
16796    0.0
16797    0.0
16798    0.0
16799    0.0
Name: 강수량(mm), Length: 16800, dtype: float64

In [75]:
test.to_csv('test_preprocessed.csv', index=False)

In [18]:
test.head()

,건물번호,기온(C),풍속(m/s),습도(%),month,day,holiday,quarter,sin_time,cos_time,DI
0,1,23.5,2.2,72,8,3,0,3,0.000000,1.000000,710.815
1,1,23.0,0.9,72,8,3,0,3,0.258819,0.965926,674.770
2,1,22.7,1.5,75,8,3,0,3,0.500000,0.866025,677.662
3,1,22.1,1.3,78,8,3,0,3,0.707107,0.707107,655.363
4,1,21.8,1.0,77,8,3,0,3,0.866025,0.500000,624.672


### Training

In [11]:
import os
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split

In [12]:
def SMAPE(true, pred):
    return np.mean((np.abs(true - pred))/(np.abs(true) + np.abs(pred))) * 100    

In [20]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,week,holiday,sin_time,cos_time,DI,power
0,1,18.6,0.0,0.9,42.0,6,2,22,0,0.000000,1.000000,234.154,1085.28
1,1,18.0,0.0,1.1,45.0,6,2,22,0,0.258819,0.965926,219.280,1047.36
2,1,17.7,0.0,1.5,45.0,6,2,22,0,0.500000,0.866025,205.672,974.88
3,1,16.7,0.0,1.4,48.0,6,2,22,0,0.707107,0.707107,167.011,953.76
4,1,18.4,0.0,2.8,43.0,6,2,22,0,0.866025,0.500000,229.592,986.40


In [13]:
def tr_ts_split(train, test, number, val_hour):
    x = train.loc[train['건물번호']==number].drop(['건물번호','power'], axis=1)
    y = train.loc[train['건물번호']==number, 'power']

    #if number == 54:
    x = pd.concat([x[:601], x[700:]], axis=0).reset_index(drop=True)
    y = pd.concat([y[:601], y[700:]]).reset_index(drop=True)

    if val_hour == 0:
        return x, None, y, None
    else:
        x_train, x_valid, y_train, y_valid = x[:-val_hour], x[-val_hour:], y[:-val_hour], y[-val_hour:]
        return x_train, x_valid.reset_index(drop=True), y_train, y_valid.reset_index(drop=True)

In [14]:
def fit_and_pred_XGB(train, test, number, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

In [19]:
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test = fit_and_pred_XGB(train, test, i+1, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:02<04:05,  2.48s/it]

SMAPE: 3.4496644441677238
R2: 0.9501395854081043
===== < BUILDING NO.2 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
  2%|▏         | 2/100 [00:03<02:32,  1.55s/it]

...

SMAPE: 6.409336428334088
R2: 0.8718903608935654
===== < BUILDING NO.99 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
 99%|█████████▉| 99/100 [02:44<00:01,  1.60s/it]

SMAPE: 1.7027544934580474
R2: 0.9782284798595801
===== < BUILDING NO.100 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
100%|██████████| 100/100 [02:46<00:00,  1.66s/it]

SMAPE: 2.6883008831880297
R2: 0.9617914179577062


In [20]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.4496644441677238, 'R2': 0.9501395854081043},
 {'건물번호': 2, 'SMAPE': 5.315951214428996, 'R2': 0.8301183990220398},
 {'건물번호': 3, 'SMAPE': 5.271764315449672, 'R2': 0.842398546048641},
 {'건물번호': 4, 'SMAPE': 1.9815124791238239, 'R2': 0.9753420265350734},
 {'건물번호': 5, 'SMAPE': 3.507672159174944, 'R2': 0.9642913768247501},
 {'건물번호': 6, 'SMAPE': 2.2468971392181447, 'R2': 0.9689029577666786},
 {'건물번호': 7, 'SMAPE': 4.333272670925453, 'R2': 0.9060958145060787},
 {'건물번호': 8, 'SMAPE': 2.44325686603149, 'R2': 0.7351378479755595},
 {'건물번호': 9, 'SMAPE': 2.174352069842542, 'R2': 0.9717021895248862},
 {'건물번호': 10, 'SMAPE': 3.623798109635635, 'R2': 0.44647936617082273},
 {'건물번호': 11, 'SMAPE': 1.6545924034733546, 'R2': 0.9039160007710692},
 {'건물번호': 12, 'SMAPE': 2.5047585567034085, 'R2': 0.8607990873955604},
 {'건물번호': 13, 'SMAPE': 3.496462420044567, 'R2': 0.19377698841111735},
 {'건물번호': 14, 'SMAPE': 8.31562679276592, 'R2': 0.27420760662127597},
 {'건물번호': 15, 'SMAPE': 1.4217463752358

In [21]:
smape_val

2.7227749700016766

In [71]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = pred_test_whole
sub.to_csv('xgb_sub_15.csv', index=False)

### Improvement

In [22]:
train['적정온도차이'] = train['기온(C)'] - 26

In [23]:
test['적정온도차이'] = test['기온(C)'] - 26

In [34]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,holiday,sin_time,cos_time,DI,power,적정온도차이
0,1,18.6,0.0,0.9,42.0,6,2,0,0.000000,1.000000,234.154,1085.28,-7.4
1,1,18.0,0.0,1.1,45.0,6,2,0,0.258819,0.965926,219.280,1047.36,-8.0
2,1,17.7,0.0,1.5,45.0,6,2,0,0.500000,0.866025,205.672,974.88,-8.3
3,1,16.7,0.0,1.4,48.0,6,2,0,0.707107,0.707107,167.011,953.76,-9.3
4,1,18.4,0.0,2.8,43.0,6,2,0,0.866025,0.500000,229.592,986.40,-7.6


In [24]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [25]:
def fit_and_pred_XGB_with_weight(train, test, number, weight, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.set_params(**{'objective':weighted_mse(weight)})
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

In [26]:
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test = fit_and_pred_XGB_with_weight(train, test, i+1, 100, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:01<03:09,  1.92s/it]

SMAPE: 3.958089806818016
R2: 0.942688600343887
===== < BUILDING NO.2 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
  2%|▏         | 2/100 [00:03<02:30,  1.54s/it]

...

SMAPE: 6.996710563035973
R2: 0.8523448293077341
===== < BUILDING NO.99 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
 99%|█████████▉| 99/100 [02:57<00:01,  1.70s/it]

SMAPE: 1.694317434400067
R2: 0.9789202633621266
===== < BUILDING NO.100 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
100%|██████████| 100/100 [02:59<00:00,  1.79s/it]

SMAPE: 2.734539119499041
R2: 0.9661826648531509


In [27]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.958089806818016, 'R2': 0.942688600343887},
 {'건물번호': 2, 'SMAPE': 5.423590043612395, 'R2': 0.8202690087147659},
 {'건물번호': 3, 'SMAPE': 5.4817390200429115, 'R2': 0.8344428371312368},
 {'건물번호': 4, 'SMAPE': 2.030205401726124, 'R2': 0.9739141290440244},
 {'건물번호': 5, 'SMAPE': 2.827683287140875, 'R2': 0.9722565011568478},
 {'건물번호': 6, 'SMAPE': 1.8545761225980184, 'R2': 0.9814595210574545},
 {'건물번호': 7, 'SMAPE': 5.067414580170736, 'R2': 0.8952315004993333},
 {'건물번호': 8, 'SMAPE': 2.5219085058843516, 'R2': 0.7352700104528325},
 {'건물번호': 9, 'SMAPE': 2.138218998898596, 'R2': 0.9743442644232198},
 {'건물번호': 10, 'SMAPE': 3.8797821999979956, 'R2': 0.33503565082129705},
 {'건물번호': 11, 'SMAPE': 1.6987581798917475, 'R2': 0.9046754964654331},
 {'건물번호': 12, 'SMAPE': 1.9467450064152463, 'R2': 0.8963663764505699},
 {'건물번호': 13, 'SMAPE': 2.9932188552135015, 'R2': 0.40865056641057806},
 {'건물번호': 14, 'SMAPE': 8.26659468656458, 'R2': 0.2823081380570013},
 {'건물번호': 15, 'SMAPE': 1.25634414101

In [28]:
smape_val

2.646947603316238

In [29]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = pred_test_whole
sub.to_csv('xgb_sub_27.csv', index=False)

### Parameter Tuning - Grid Search CV

In [48]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [ ]:
df = pd.DataFrame(columns = ['n_estimators', 'eta', 'min_child_weight', 'max_depth', 'colsample_bytree', 'subsample'])
preds = np.array([])
grid = {
    'n_estimators':     [30, 50, 70, 100],
    'eta':              [0.01],
    'min_child_weight': np.arange(1, 8, 1),
    'max_depth':        np.arange(3, 9, 1),
    'colsample_bytree': np.arange(0.8, 1.0, 0.1),
    'subsample':        np.arange(0.8, 1.0, 0.1)
}

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    gcv = GridSearchCV(estimator=XGBRegressor(seed=0, gpu_id=0,
                                              tree_method='gpu_hist', predictor='gpu_predictor'),
                       param_grid=grid,
                       scoring=make_scorer(SMAPE, greater_is_better=False),
                       cv=5,
                       refit=True,
                       verbose=True)
    gcv.fit(x_train, y_train)
    best = gcv.best_estimator_
    params = gcv.best_params_
    print(params)
    pred = best.predict(x_valid)
    print(f'SMAPE: {SMAPE(y_valid, pred)}')
    preds = np.append(preds, pred)
    df = pd.concat([df, pd.DataFrame(params, index=[0])], axis=0)

  0%|          | 0/100 [00:00<?, ?it/s]

===== < BUILDING NO.1 > =====
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


In [133]:
df.to_csv('hyperparameter_xgb.csv', index=False)

In [ ]:
xgb_params = df

In [ ]:
scores = []
best_iter = []

for i in tqdm(range(100)):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                            max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4],
                            subsample=xgb_params.iloc[i, 5], seed=0)
    xgb_reg.set_params(**{'objective':weighted_mse(100)})
    xgb_reg.set_params(early_stopping_rounds=300)
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], verbose=False)
    y_pred = xgb_reg.predict(x_valid)
    pred = pd.Series(y_pred)
    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    best_iter.append(xgb_reg.best_iteration) # 실제 best iteration은 이 값에 +1

In [ ]:
alpha_list = []
smape_list = []

for i in tqdm(range(100)):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    xgb = XGBRegressor(seed=0,
                        n_estimators=best_iter[i], eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                        max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
    xgb.fit(x_train, y_train)
    pred0 = xgb.predict(x_valid)
    best_alpha = 0
    score0 = SMAPE(y_valid, pred0)

    for j in [1, 2, 5, 7, 10, 25, 50, 75, 100]:
        xgb = XGBRegressor(seed=0,
                        n_estimators=best_iter[i], eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                        max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
        xgb.set_params(**{'objective': weighted_mse(j)})
        xgb.fit(x_train, y_train)
        pred1 = xgb.predict(x_valid)
        score1 = SMAPE(y_valid, pred1)
        if score1 < score0:
            best_alpha = j
            score0 = score1

    alpha_list.append(best_alpha)
    smape_list.append(score0)
    print(f"building {i+1} || best score: {score0} || alpha: {best_alpha}")

In [ ]:
xgb_params['alpha'] = alpha_list
xgb_params['best_iter'] = best_iter
xgb_params.head()

In [ ]:
xgb_params.to_csv('hyperparameter_xgb_real_last_testing.csv', index=False)

In [ ]:
best_iter = xgb_params['best_iter'].to_list()
best_iter[0]

In [212]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,week,holiday,sin_time,cos_time,DI,power,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),건물유형_건물기타,건물유형_공공,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트,CDH
0,1,18.6,0.0,0.9,42.0,6,2,22,0,0.000000,1.000000,234.154,1085.28,110634.0,39570.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,-7.4
1,1,18.0,0.0,1.1,45.0,6,2,22,0,0.258819,0.965926,219.280,1047.36,110634.0,39570.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,-15.4
2,1,17.7,0.0,1.5,45.0,6,2,22,0,0.500000,0.866025,205.672,974.88,110634.0,39570.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,-23.7
3,1,16.7,0.0,1.4,48.0,6,2,22,0,0.707107,0.707107,167.011,953.76,110634.0,39570.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,-33.0
4,1,18.4,0.0,2.8,43.0,6,2,22,0,0.866025,0.500000,229.592,986.40,110634.0,39570.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,-40.6


In [ ]:
preds = np.array([])
for i in tqdm(range(100)):
    pred_df = pd.DataFrame()
    for seed in range(0, 6):
        x_train = train.loc[train['건물번호']==i+1,].drop(['건물번호', 'power'], axis=1)
        y_train = train.loc[train['건물번호']==i+1, 'power']
        x_test = test.loc[test['건물번호']==i+1,].drop('건물번호', axis=1)

        xgb = XGBRegressor(seed=seed, n_estimators=best_iter[i], eta=0.01,
                            min_child_weight=xgb_params.iloc[i, 2], max_depth=xgb_params.iloc[i, 3],
                            colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
        if xgb_params.iloc[i, 6] != 0:
            xgb.set_params(**{'objective':weighted_mse(xgb_params.iloc[i, 6])})

        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        pred_df.loc[:, seed] = y_pred

    pred = pred_df.mean(axis=1)
    preds = np.append(preds, pred)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = preds
sub.to_csv('xgb_sub_22.csv', index=False)

In [215]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.5419388838477404, 'R2': 0.9563832166343148},
 {'건물번호': 2, 'SMAPE': 4.295780152455329, 'R2': 0.8651174662607124},
 {'건물번호': 3, 'SMAPE': 4.976127431146375, 'R2': 0.8282886548210623},
 {'건물번호': 4, 'SMAPE': 1.916974758959043, 'R2': 0.9749491993399805},
 {'건물번호': 5, 'SMAPE': 2.8188883844313564, 'R2': 0.972446435735248},
 {'건물번호': 6, 'SMAPE': 1.8247299710309055, 'R2': 0.9802984729616211},
 {'건물번호': 7, 'SMAPE': 4.181533402777831, 'R2': 0.906970899840581},
 {'건물번호': 8, 'SMAPE': 2.8739138223687473, 'R2': 0.6702282399188223},
 {'건물번호': 9, 'SMAPE': 1.8549493161639723, 'R2': 0.984736771457575},
 {'건물번호': 10, 'SMAPE': 3.5416225095446876, 'R2': 0.4322418016566194},
 {'건물번호': 11, 'SMAPE': 1.686647477272509, 'R2': 0.8955227858960689},
 {'건물번호': 12, 'SMAPE': 1.9311341180054387, 'R2': 0.8995117473179703},
 {'건물번호': 13, 'SMAPE': 3.004711373509035, 'R2': 0.3961679661426334},
 {'건물번호': 14, 'SMAPE': 7.909728524427743, 'R2': 0.34326299780211256},
 {'건물번호': 15, 'SMAPE': 1.8971932943063

In [57]:
train[train['건물번호']==54].head(24)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
108120,54_20220601 00,54,20220601 00,18.6,NaN,2.6,62.0,NaN,NaN,578.16
108121,54_20220601 01,54,20220601 01,18.8,NaN,0.4,62.0,NaN,NaN,408.24
108122,54_20220601 02,54,20220601 02,18.9,NaN,0.0,52.0,NaN,NaN,398.16
108123,54_20220601 03,54,20220601 03,20.2,NaN,2.4,38.0,NaN,NaN,387.36
108124,54_20220601 04,54,20220601 04,20.4,NaN,2.8,40.0,NaN,NaN,387.36
108125,54_20220601 05,54,20220601 05,20.0,NaN,1.7,41.0,NaN,NaN,419.76
108126,54_20220601 06,54,20220601 06,19.6,NaN,0.0,47.0,0.0,0.04,612.72
108127,54_20220601 07,54,20220601 07,21.2,NaN,2.0,38.0,0.0,0.29,661.68
108128,54_20220601 08,54,20220601 08,22.9,NaN,1.3,36.0,0.0,0.69,1210.32
108129,54_20220601 09,54,20220601 09,25.6,NaN,0.3,32.0,0.8,1.43,1901.16


In [52]:
bi = pd.read_csv('building_info.csv')

In [54]:
bi[bi['건물번호']==54]

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
53,54,상용,109400.2,65803.57,-,-,-


test와 train의 분포 차이 확인
random forest 모델 확인
과대 예측 잡기
54번 건물 주의(튀는 애 발견, outlier 제거 후 시도 예를 들면 뒤에서 1000개 빼서 수행)
feature selection 요망